In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from skimage import io , transform
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms , utils

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
!mkdir '../../kaggle/working/resnet18'
!mkdir '../../kaggle/working/resnet18/models'
!mkdir '../../kaggle/working/resnet18/models/check'
!mkdir '../../kaggle/working/resnet18/models/best'


In [ ]:
#Hyperparameters
n_epochs = 3
#batch_train = 64
#batch_test  = 1000

batch_size=16

learning_rate = 1e-4 
momentum = 0.9
log_interval = 10            #printing logs after an interval

random_seed = 32
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Neptune
! pip install neptune-client==0.4.132

In [ ]:
import neptune

neptune.init(project_qualified_name='manickavela/computer-vision', # change this to your `workspace_name/project_name`
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiN2JhMDRjODUtZjk1Ni00M2JjLWI3ZTctYTg5NmFlOGNlOTNmIn0=', # change this to your api token
            )


#project = neptune.init('manickavela/computer-vision',
#                       api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiN2JhMDRjODUtZjk1Ni00M2JjLWI3ZTctYTg5NmFlOGNlOTNmIn0=')
#my_neptune = project.get_experiments(id='COM-4')[0]


In [ ]:
!ls '../../kaggle/input/mura-v11/MURA-v1.1'

In [ ]:
train_image_path  = pd.read_csv("../../kaggle/input/mura-v11/MURA-v1.1/train_image_paths.csv",names=["path"])
valid_image_path  = pd.read_csv("../../kaggle/input/mura-v11/MURA-v1.1/valid_image_paths.csv",names=["path"])

In [ ]:
my_list = []

for i in range(len(train_image_path)) :
    my_list.append(re.search('/train/(.*)/patient',train_image_path['path'].iloc[i]).group(1))

my_list = list(set(my_list))
my_list

In [ ]:
classes = my_list

enum = enumerate(classes)
ClassToLabel = dict((i,j) for i,j in enum)
ClassToLabel
#LabelToClass = dict((b,a) for a,b in ClassToLabel)
#LabelToClass

In [ ]:
classes = my_list

enum = enumerate(classes)
LabelToClass = dict((b,a) for a,b in enum)
LabelToClass

In [ ]:
class ScannedDataset(Dataset) :
    def __init__(self,csv_file,root_dir,train=True,transform=None) :
        self.paths = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.train = train
    def __len__(self) :
        return len(self.paths)

    def __getitem__(self,idx) :
        if torch.is_tensor(idx) :
            idx = idx.tolist()
        #print('Path feeded : ',self.paths.iloc[idx,0])
        img_name = os.path.join(self.root_dir,self.paths.iloc[idx,0])

        image = io.imread(img_name)
        if self.train == True :
            label = re.search('/train/(.*)/patient',img_name).group(1)
        elif self.train == False :
            label = re.search('/valid/(.*)/patient',img_name).group(1)
        #print(label)
        label = LabelToClass[label]
        #print("sample : ",label,' ',image.shape)
        if len(image.shape) > 2 :
            if image.shape[2] == 3 :
                image = (image[:,:,0] + image[:,:,1] + image[:,:,2])/3
                #print('Made into grayscale ')
        sample = {'image':image , 'labels' : label}

        if self.transform :
            sample = self.transform(sample)

        
        return sample



In [ ]:
class Rescale(object) :
    def __init__(self,output_size) :
        self.output_size = output_size

    def __call__(self,sample) :
        image , labels = sample['image'] , sample['labels']

        h , w = image.shape[:2] 
        new_h , new_w = self.output_size
        new_h , new_w = int(new_h) , int(new_w)

        #if len(image.shape) > 2 :                                               #Ignoring 3rd channel
        #    return {'image': None , 'labels' : None}

        img = transform.resize(image , (new_h , new_w))
        #print('Befroe sending : ',labels,' ',img)
        return {'image' :img , 'labels' : labels}

class ToTensor(object) :
    def __call__(self,sample) :
        #print('I came here')
        image , labels = sample['image'] , sample['labels']

        #print('Length : ',len(image.shape))
        if len(image.shape) < 3 :
            image = image[:,:,np.newaxis]
        # swap color axis because

        # numpy image: H x W x C
        # torch image: C X H X W
        #print('Imgae shape before reshaping : ',image.shape)
        image = image.transpose((2,0,1))
        return {'image':torch.from_numpy(image).float() , 'labels' : labels}

In [ ]:
my_dataset = ScannedDataset(csv_file='../../kaggle/input/mura-v11/MURA-v1.1/train_image_paths.csv',
                            root_dir='../../kaggle/input/mura-v11/',
                            train=True,
                            transform=transforms.Compose([Rescale((512,512)) , ToTensor()])) 

my_test = ScannedDataset(csv_file='../../kaggle/input/mura-v11//MURA-v1.1/valid_image_paths.csv',
                            root_dir='../../kaggle/input/mura-v11/',
                            train=False,
                            transform=transforms.Compose([Rescale((512,512)) , ToTensor()])) 
'''
for i in range(len(my_dataset)) :
    sample = my_dataset[i]
    #print(i,'', sample['image'].shape,'', sample['labels'])
    
    if i == 3:
        break
'''

In [ ]:
train_loader = torch.utils.data.DataLoader(my_dataset,batch_size = batch_size,shuffle=True,num_workers=4)
test_loader  = torch.utils.data.DataLoader(my_test,batch_size = batch_size,shuffle=True)

In [ ]:
def conv3x3(in_planes,out_planes,stride=1,groups=1,dilation=1) :
    #in_planes = 1
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,
                     padding=dilation,groups=groups,bias=False,dilation=dilation)
    
def conv1x1(in_planes,out_planes,stride=1) :
    return nn.Conv2d(in_planes,out_planes,kernel_size=1,stride=stride,bias=False)

In [ ]:
class BasicBlock(nn.Module) :
    expansion = 1
    def __init__(self,inplanes,planes,stride=1,downsample=None,groups=1,
                 base_width=64,dilation=1,norm_layer=None) :
        super(BasicBlock,self).__init__()
        if norm_layer is None :
            norm_layer = nn.BatchNorm2d
        
        self.conv1 = conv3x3(inplanes,planes,stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes,planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self,x) :
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None :
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class Bottleneck(nn.Module) :
    expansion = 4

    def __init__(self,inplanes,planes,stride=1,downsample=None,groups=1,
                 base_width=64,dilation=1,norm_layer=None) :
                super(Bottleneck,self).__init__()

                if norm_layer is None :
                     norm_layer = nn.BotchNorm2d

                width = int(planes * (base_width/64.))

                self.conv1 = conv1x1(inplanes,width)
                self.bn1 = norm_layer(width)
                self.conv2 = conv3x3(width, width, stride, groups, dilation)
                self.bn2 = norm_layer(width)
                self.conv3 = conv1x1(width, planes * self.expansion)
                self.bn3 = norm_layer(planes * self.expansion)
                self.relu = nn.ReLU(inplace=True)
                self.downsample = downsample
                self.stride = stride
    def forward(self,x) :
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module) :
    def __init__(self,block,layers,num_classes=1000,zero_init_residual=False,
                 groups=1,width_per_group=6,replace_stride_with_dilation=None,norm_layer=None) :
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  
                    nn.init.constant_(m.bn2.weight, 0)  

    def _make_layer(self, block, planes, blocks,
                    stride= 1, dilate = False) :
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation

        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x) :
        #print('ResNet : ',x.shape)
        #print('Mess from here\n')
        x = self.conv1(x)
        #print('Mess over\n')
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x) :
        #print('Herere too\n',x)
        return self._forward_impl(x)
    
def _resnet(arch: str,block,layers,pretrained: bool, progress: bool,**kwargs) :            
    model = ResNet(block, layers, **kwargs)
    #if pretrained:
    #    state_dict = load_state_dict_from_url(model_urls[arch],
    #                                          progress=progress)
    #    model.load_state_dict(state_dict)
    return model


In [ ]:

def resnet18(pretrained: bool = False, progress: bool = True, **kwargs) :

    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

In [ ]:
#Creating ResNet
'''
net_args = {
    "block" : ResidualBlock,
    "layers" : [2,2,2,2]
}
'''
#network = ResNet(**net_args)
network = resnet18().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(network.parameters(),lr = learning_rate,momentum=momentum)

neptune.create_experiment('MURA Resnet18',params={'Learning Rate':learning_rate,'Momentum':momentum,'Model architecture ':'Resnet18','Epoch':50,
                                                 'Optimizer':'SGD','Criterion':'Cross Entropy Loss','Batch Size':batch_size})


In [ ]:
optimizer

In [ ]:
import shutil

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    f_path = checkpoint_path
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path, best_fpath)


In [ ]:
#../../kaggle/input/mura-v11/
checkpoint_path = "../../kaggle/working/resnet18/models/check/check_point_"
bestmodel_path  = "../../kaggle/working/resnet18/models/best/best_model_"

In [ ]:
%%time
train_count = 0
test_count = 0

train_losses = []
valid_losses = []
accuracy_train = []
accuracy_valid = []

correct_train = 0
correct_test = 0

#neptune.log_metric('Epoch',20)
valid_loss_min = 100

for epoch in range(1,50) :
    train_loss = 0.0
    valid_loss = 0.0

    train_acc = 0.0
    valid_acc = 0.0

    correct_train = 0
    correct_test = 0

    #Training
    network.train()
    print('Training...')
    size = 0
    for i,data in enumerate(train_loader) :

        print('Batches : ',i) 
        inputs, labels = data['image'] , data['labels']
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() 
        output = network(inputs)

        loss = criterion(output,labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()*inputs.size(0)

        pred = output.data.max(1,keepdim=True)[1]
        correct_train += pred.eq(labels.data.view_as(pred)).sum() 
        size += len(pred)
        print('corr : ',correct_train)
        train_acc = correct_train/size          #len(pred) has 

        #print(len(pred) )
        print('Train accuracy ',train_acc)
        print('Train Loss ',loss.item())

        accuracy_train.append(train_acc)
        train_losses.append(loss.item())

        neptune.log_metric('train_loss',loss.item())
        neptune.log_metric('train_acc',train_acc)

        train_count = i
    #train_acc = 100. * correct_train/len(train_loader.dataset)
    #print('Accuracy : ',100. * correct_train/len(train_loader.dataset))
    #accuracy_train.append(100. * correct_train/len(train_loader.dataset))
    
    
    #Validataion
    print('Validating...')
    size = 0
    network.eval()
    with torch.no_grad() : 
        for i,data in enumerate(test_loader) :
            print('Batches : ',i) 
       
            target = data['labels']
            data = data['image']

            data = data.to(device)
            target = target.to(device)

            output = network(data)

            loss = criterion(output,target)

            valid_loss += loss.item()*data.size(0)

            pred = output.data.max(1,keepdim=True)[1]
            correct_test += pred.eq(target.data.view_as(pred)).sum() 
            size += len(pred)
            valid_acc = correct_test/size

            accuracy_valid.append(valid_acc)
            valid_losses.append(loss.item())
        
            neptune.log_metric('valid_loss',loss.item())
            neptune.log_metric('valid_acc',valid_acc)

            test_count = i
        
        #print('Valid acc  : ',valid_acc)
        #print('Valid loss : ',loss.item())
    #valid_acc = 100. * correct_test/len(test_loader.dataset)
    #print('Accuracy : ',100. * correct_test/len(test_loader.dataset))
    #accuracy_test.append(100. * correct_test/len(test_loader.dataset))
    

    checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': network.state_dict(),
            'optimizer': optimizer.state_dict(),
        }

    #save_ckp(checkpoint, False, checkpoint_path +'.pt', bestmodel_path+str(epoch+1))

    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(test_loader.sampler)
    #train_losses.append(train_loss)
    #valid_losses.append(valid_loss)

    neptune.log_metric('train_loss_epoch',train_loss)
    neptune.log_metric('valid_loss_epoch',valid_loss)

    neptune.log_metric('train_acc_epoch',train_acc)
    neptune.log_metric('valid_acc_epoch',valid_acc)

    #Saving the best model till now 
    if valid_loss <= valid_loss_min:
            #print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            save_ckp(checkpoint, True, checkpoint_path+str(epoch+1)+'.pt', bestmodel_path +str(epoch+1)+'.pt')
            valid_loss_min = valid_loss
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

In [ ]:
mkdir '../../kaggle/working/resnet18'
mkdir '../../kaggle/working/resnet18/models'
mkdir '../../kaggle/working/resnet18/models/check'
mkdir '../../kaggle/working/resnet18/models/best'


In [ ]:
ls  g